In [93]:
import os
import requests
from dotenv import load_dotenv

# Charger la clé API depuis .env pour éviter de l'exposer dans le notebook
load_dotenv()
api_key = os.getenv("OPENROUTER_API_KEY")
if not api_key:
    raise ValueError("Merci d'ajouter OPENROUTER_API_KEY dans ton fichier .env avant d'exécuter ce notebook.")

# 🔍 Prompt de test
prompt = "1+2=???."

# Corps de la requête
payload = {
    "model": "gpt-4o-mini",
    "messages": [
        {"role": "system", "content": "Tu es un assistant francophone."},
        {"role": "user", "content": prompt}
    ],
    "temperature": 0.5,
    "max_tokens": 500
}

# Envoi de la requête à OpenRouter
response = requests.post(
    "https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    },
    json=payload
)

# Vérification et affichage
if response.ok:
    data = response.json()
    print("=== Réponse du modèle ===\n")
    print(data["choices"][0]["message"]["content"])
else:
    print(f"❌ Erreur {response.status_code}: {response.text}")

=== Réponse du modèle ===

1 + 2 = 3.


## RAG project

## step 1:  Collecte des données

In [38]:
from PyPDF2 import PdfReader

reader = PdfReader("Cours_IA_et_Applications_S1.pdf")
text = " ".join(page.extract_text() for page in reader.pages)


## step 2 : Preprocess and Split Documents

In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(text)

## step 3 :  Create Embeddings

In [40]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: baa8fdeb-7efb-4cab-93c6-81211350eb6b)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


## 5 Store Vectors in a Vector Database

In [41]:
import faiss
import numpy as np

dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype('float32'))
index.ntotal

31

## 5 Build Retrieval Logic

In [78]:
query_embedding = model.encode(["expliquer Défis de l’IA dans le cours"])
D, I = index.search(np.array(query_embedding).astype('float32'), k=5)
retrieved_chunks = [chunks[i] for i in I[0]]

## Step 6: Generate Answer with LLM

In [89]:
# Suppose these are the retrieved chunks
# retrieved_chunks = [
#     "Introduction à l'IA et ses applications",
#     "Apprentissage automatique de base"
# ]

# Combine chunks + user question
prompt = f"Voici des informations du cours :\n" + "\n".join(retrieved_chunks) + "\n\nQuestion : apartir de ces informations, donner un resume  ?"

payload = {
    "model": "gpt-4o-mini",
    "messages": [
        {"role": "system", "content": "Tu es un expert en IA et science des données."},
        {"role": "user", "content": prompt}
    ],
    "temperature": 0.5,
    "max_tokens": 200
}
response = requests.post(
    "https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    },
    json=payload
)

In [88]:
retrieved_chunks

['×Présence en cours/TD/TP (corps et esprit)\n×Respecter le droit des autres à écouter : pas de bavardage en classe !\n\uf0fcMobiliser toutes les sources d’apprentissage : écouter, prendre des notes, etc.\n\uf0fcQuestions à volonté (cours/TD/TP) !\n\uf0fcFaire les devoirs et les rendre dans les délais\n 21/09/2025 IA et Applications 4Elément 1 : IA et Applications: ~3H30 de cours : Cours \nElément 2: Projet: ~1H50 de TP (par groupe) :  Réponse aux questions, travail sur PCOrganisation et Evaluation\nElément 1 :',
 'triste?Pourquoi vous\nsentez -vous triste\nJe suis au travail Travail: parlez un \npeu de votre travailparlez un peu de \nvotre travail\nJe suis triste parce \nque j’ai échoué \nmon examen.? ? 21/09/2025 IA et Applications 15Définition et Contexte\n1950 1960 -1970 1980 -1990\nAlan Turing (1950) : Test \nde Turing → une \nmachine est dite \nintelligente si un \nhumain ne peut pas \ndistinguer ses réponses \nde celles d’un autre \nhumain dans une \nconversation.L’IA symbolique

In [ ]:
# Vérification et affichage
if response.ok:
    data = response.json()
    print("=== Réponse du modèle ===\n")
    print(data["choices"][0]["message"]["content"])
else:
    print(f"❌ Erreur {response.status_code}: {response.text}")

=== Réponse du modèle ===

Le cours sur l'Intelligence Artificielle (IA) et ses applications se déroule en plusieurs éléments, incluant des cours théoriques et des travaux pratiques. Les étudiants sont encouragés à être présents et attentifs, à respecter les autres, à poser des questions et à rendre leurs devoirs à temps. 

Le premier élément du cours se concentre sur les bases de l'IA, en commençant par la définition de l'intelligence selon Alan Turing, qui a introduit le Test de Turing en 1950. Ce test détermine si une machine peut être considérée comme intelligente si ses réponses ne peuvent être distinguées de celles d'un humain. L'IA symbolique est également abordée, avec des exemples tels que le General Problem Solver (GPS) et le chatbot ELIZA, qui simule une interaction psychothérapeutique.

Les étudiants apprendront à définir un problème d'IA supervisée, en précisant l'objectif principal, les


In [ ]:
# =====================================================
# 1️⃣ Imports et configuration
# =====================================================
import os
from dotenv import load_dotenv
from PyPDF2 import PdfReader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import BM25Retriever

load_dotenv()

# 🔑 Clé OpenRouter
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise ValueError("Merci de mettre ta clé OPENROUTER_API_KEY dans le fichier .env")
# =====================================================
# 2️⃣ Chargement et extraction de tous les cours PDF
# =====================================================
folder_path = "data/"  # dossier contenant tous les PDFs
raw_documents = []

for filename in os.listdir(folder_path):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(folder_path, filename)
        reader = PdfReader(pdf_path)
        page_texts = []
        for page in reader.pages:
            content = (page.extract_text() or "").replace("\n", " ").strip()
            if content:
                page_texts.append(content)
        full_text = " ".join(page_texts)
        if full_text:
            raw_documents.append({"text": full_text, "metadata": {"source": filename}})

if not raw_documents:
    raise ValueError("Aucun texte extrait des PDFs. Vérifie que le dossier data/ contient des fichiers PDF lisibles.")

print(f"✅ {len(raw_documents)} cours PDF exploitables.")
# =====================================================
# 3️⃣ Découpage des textes en chunks pour le retrieval
# =====================================================
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=350,
    chunk_overlap=120
)

documents = []
for item in raw_documents:
    splitted_docs = text_splitter.create_documents([item["text"]], metadatas=[item["metadata"]])
    for idx, doc in enumerate(splitted_docs):
        doc.metadata["chunk_index"] = idx
    documents.extend(splitted_docs)

if not documents:
    raise ValueError("Aucun chunk généré. Vérifie les textes extraits.")
print(f"✅ Total chunks indexés : {len(documents)}")
# =====================================================
# 4️⃣ Création des embeddings multilingues normalisés
# =====================================================
embedding_model = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    encode_kwargs={"normalize_embeddings": True}
)

vectorstore = FAISS.from_documents(documents, embedding_model)
print("✅ Index FAISS construit avec embeddings e5.")
# =====================================================
# 5️⃣ Configurer un retriever hybride (cosinus + BM25)
# =====================================================
dense_retriever = vectorstore.as_retriever(
    search_type="mmr",  # Maximum Marginal Relevance pour diversité
    search_kwargs={"k": 3, "fetch_k": 4}
)

sparse_retriever = BM25Retriever.from_documents(documents)
sparse_retriever.k = 8

hybrid_retriever = EnsembleRetriever(
    retrievers=[dense_retriever, sparse_retriever],
    weights=[0.65, 0.35]
)
# =====================================================
# 6️⃣ Configurer le modèle GPT-4o-mini via OpenRouter
# =====================================================
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.4,  # température plus basse pour des réponses plus factuelles
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",
    default_headers={
        "HTTP-Referer": "https://github.com/your-user/your-repo",
        "X-Title": "RAG Notebook",
    }
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=hybrid_retriever,
    return_source_documents=True,
    chain_type="stuff"
)
# =====================================================
# 7️⃣ Exemple : rechercher un chapitre et générer une réponse
# =====================================================
query = "Expliquer les principaux défis de l’intelligence artificielle"

result = qa_chain(query)

print("=== Réponse du modèle ===\n")
print(result["result"])

print("\n=== Sources utilisées ===\n")
for i, doc in enumerate(result["source_documents"]):
    print(f"--- Source {i} ({doc.metadata.get('source')}, chunk {doc.metadata.get('chunk_index')}) ---")
    print(doc.page_content[:300], "...\n")
# =====================================================
# 8️⃣ Exemple : générer un mini quiz en français
# =====================================================
query_quiz = """
Génère 5 questions de quiz avec leurs réponses à partir des passages pertinents
des cours sur l’intelligence artificielle.
"""

result_quiz = qa_chain(query_quiz)

print("=== Quiz généré ===\n")
print(result_quiz["result"])

✅ 1 cours PDF exploitables.
✅ Total chunks indexés : 61


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

✅ Index FAISS construit avec embeddings e5.


ImportError: Could not import rank_bm25, please install with `pip install rank_bm25`.

In [105]:
dense_retriever = vectorstore.as_retriever(
    search_type="mmr",  # Maximum Marginal Relevance pour diversité
    search_kwargs={"k": 3, "fetch_k": 4}
)
sparse_retriever = BM25Retriever.from_documents(documents)
sparse_retriever.k = 3

hybrid_retriever = EnsembleRetriever(
    retrievers=[dense_retriever, sparse_retriever],
    weights=[0.65, 0.35]
)
# =====================================================
# 6️⃣ Configurer le modèle GPT-4o-mini via OpenRouter
# =====================================================
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.4,  # température plus basse pour des réponses plus factuelles
    openai_api_key=OPENROUTER_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",
    default_headers={
        "HTTP-Referer": "https://github.com/your-user/your-repo",
        "X-Title": "RAG Notebook",
    }
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=hybrid_retriever,
    return_source_documents=True,
    chain_type="stuff"
)
# =====================================================
# 7️⃣ Exemple : rechercher un chapitre et générer une réponse
# =====================================================
query = "donner 5 principaux défis de l’intelligence artificielle"

result = qa_chain(query)

print("=== Réponse du modèle ===\n")
print(result["result"])

print("\n=== Sources utilisées ===\n")
for i, doc in enumerate(result["source_documents"]):
    print(f"--- Source {i} ({doc.metadata.get('source')}, chunk {doc.metadata.get('chunk_index')}) ---")
    print(doc.page_content[:300], "...\n")
# =====================================================
# 8️⃣ Exemple : générer un mini quiz en français
# =====================================================
# query_quiz = """
# Génère 5 questions de quiz avec leurs réponses à partir des passages pertinents
# des cours sur l’intelligence artificielle.
# """

# result_quiz = qa_chain(query_quiz)

# print("=== Quiz généré ===\n")
# print(result_quiz["result"])

=== Réponse du modèle ===

Les cinq principaux défis de l'intelligence artificielle sont :

1. **Qualité et quantité des données** : L'IA nécessite des données propres, complètes et représentatives pour fonctionner efficacement. Des données biaisées ou incomplètes peuvent tromper les modèles.

2. **Coût computationnel et énergétique** : Certains modèles d'IA nécessitent des ressources matérielles et énergétiques importantes, ce qui peut limiter leur accessibilité et leur durabilité.

3. **Difficultés liées aux données ou à l’environnement** : Les environnements changeants ou les données instables peuvent poser des défis pour le déploiement et l'efficacité des systèmes d'IA.

4. **Limites de l’IA pour résoudre des problèmes complexes** : L'IA peut avoir des difficultés à traiter des problèmes qui nécessitent une compréhension profonde ou des jugements nuancés.

5. **Exploitation de failles dans la prise de décision automatisée** : Les systèmes d'IA peuvent être trompés par des manipulat

In [102]:

# =====================================================
# 7️⃣ Exemple : rechercher un chapitre et générer une réponse
# =====================================================
query = "Expliquer les principaux défis de l’intelligence artificielle"

result = qa_chain(query)

print("=== Réponse du modèle ===\n")
print(result["result"])

print("\n=== Sources utilisées ===\n")
for i, doc in enumerate(result["source_documents"]):
    print(f"--- Source {i} ({doc.metadata.get('source')}, chunk {doc.metadata.get('chunk_index')}) ---")
    print(doc.page_content[:300], "...\n")
# =====================================================
# 8️⃣ Exemple : générer un mini quiz en français
# =====================================================
query_quiz = """
Génère 5 questions de quiz avec leurs réponses à partir des passages pertinents
des cours sur l’intelligence artificielle.
"""

result_quiz = qa_chain(query_quiz)

print("=== Quiz généré ===\n")
print(result_quiz["result"])

=== Réponse du modèle ===

Les principaux défis de l'intelligence artificielle incluent :

1. **Qualité et quantité des données** : L'IA dépend fortement de données propres, complètes et pertinentes. Des données de mauvaise qualité peuvent entraîner des résultats erronés.

2. **Limites de l'IA** : Les modèles peuvent échouer à généraliser sur des données nouvelles ou inattendues, ce qui peut limiter leur efficacité dans des situations réelles.

3. **Explicabilité et interprétabilité** : Les modèles complexes, comme les réseaux de neurones profonds, sont souvent considérés comme des "boîtes noires", rendant difficile la compréhension des décisions qu'ils prennent. Cela peut poser des problèmes, notamment dans des domaines critiques comme la santé.

4. **Coût computationnel et énergétique** : Certains modèles d'IA nécessitent des ressources matérielles et énergétiques importantes, ce qui peut être un obstacle à leur déploiement à grande échelle.

5. **Difficultés liées aux données ou à l